In [3]:
!pip install tensorflow
!pip install scikit-learn
!pip install scikeras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
   ---------------------------------------- 1.8/331.9 MB 16.7 MB/s eta 0:00:20
   - -------------------------------------- 9.2/331.9 MB 27.2 MB/s eta 0:00:12
   -- ------------------------------------- 17.0/331.9 MB 31.6 MB/s eta 0:00:10
   --- ------------------------------------ 26.0/331.9 MB 34.3 MB/s eta 0:00:09
   ---- ----------------------------------- 34.3/331.9 MB 35.2 MB/s eta 0:00:09
   ----- ---------------------------------- 43.8/331.9 MB 36.7 MB/s eta 0:00:08
   ------ --------------------------------- 53.2/331.9 MB 37.7 MB/s eta 0:00:08
   ------- -------------------------------- 62.1/331.9 MB 38.1 MB/s eta 0:00:08
   -------- ------------------------------- 71.3/331.9 MB 38.5 MB/s eta 0:00:07
   --------- ------------------------------ 76.0/331.9 MB 36.7 MB/s eta 0:00:07
   ---------- ----------------------------- 85.2/331.9 MB 37.2 MB/s eta 0:00:07
   ----------- ---------------------------- 94.6/33

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.32.1 which is incompatible.


In [4]:
# Load the dataset
try:
    df = pd.read_csv('Alphabets_data.csv')
except FileNotFoundError:
    print("Error: Alphabets_data.csv not found. Please ensure the file is in the correct directory.")
    exit()

In [5]:
# Data Exploration
print("--- Data Exploration ---")
print("Dataset shape:", df.shape)
print("\nColumn information:")
print(df.info())
print("\nFirst 5 rows of the dataset:")
print(df.head())
print("\nDescriptive statistics:")
print(df.describe())


--- Data Exploration ---
Dataset shape: (20000, 17)

Column information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  20000 non-null  object
 1   xbox    20000 non-null  int64 
 2   ybox    20000 non-null  int64 
 3   width   20000 non-null  int64 
 4   height  20000 non-null  int64 
 5   onpix   20000 non-null  int64 
 6   xbar    20000 non-null  int64 
 7   ybar    20000 non-null  int64 
 8   x2bar   20000 non-null  int64 
 9   y2bar   20000 non-null  int64 
 10  xybar   20000 non-null  int64 
 11  x2ybar  20000 non-null  int64 
 12  xy2bar  20000 non-null  int64 
 13  xedge   20000 non-null  int64 
 14  xedgey  20000 non-null  int64 
 15  yedge   20000 non-null  int64 
 16  yedgex  20000 non-null  int64 
dtypes: int64(16), object(1)
memory usage: 2.6+ MB
None

First 5 rows of the dataset:
  letter  xbox  ybox  width  height  onpix  xbar

In [6]:
# Data Preprocessing
print("\n--- Data Preprocessing ---")
# Separate features and target
X = df.drop('letter', axis=1)
y = df['letter']


--- Data Preprocessing ---


In [7]:
# Encode the target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)
print("\nUnique classes (encoded):", np.unique(y_encoded))
print("Original classes:", le.inverse_transform(np.unique(y_encoded)))


Unique classes (encoded): [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]
Original classes: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


In [8]:
# Normalize the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("\nFeatures normalized.")


Features normalized.


In [9]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)
print("\nData split into training and test sets.")
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")


Data split into training and test sets.
Training set size: 16000 samples
Test set size: 4000 samples


In [10]:
# Function to create a basic ANN model
def create_model(layers=1, neurons=64, activation='relu'):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation=activation))
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation=activation))
    model.add(Dense(len(le.classes_), activation='softmax')) # Output layer for classification
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
# Train and evaluate a basic model with default hyperparameters
print("\n--- Training Basic Model ---")
default_model = create_model()
default_model.summary()

history = default_model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)


--- Training Basic Model ---


C:\Users\vishu_pdk4f5i\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │           1,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 26)                  │           1,690 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,778 (10.85 KB)

 Trainable params: 2,778 (10.85 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.2797 - loss: 2.9579 - val_accuracy: 0.4444 - val_loss: 2.5604
Epoch 2/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4837 - loss: 2.2271 - val_accuracy: 0.5531 - val_loss: 1.9327
Epoch 3/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5688 - loss: 1.7779 - val_accuracy: 0.5975 - val_loss: 1.6366
Epoch 4/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6089 - loss: 1.5482 - val_accuracy: 0.6369 - val_loss: 1.4626
Epoch 5/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6331 - loss: 1.4076 - val_accuracy: 0.6494 - val_loss: 1.3595
Epoch 6/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6559 - loss: 1.3147 - val_accuracy: 0.6513 - val_loss: 1.2838
Epoch 7/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6727 - loss: 1.2430 - val_accuracy: 0.6862 - val_loss: 1.2215
Epoch 8/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6887 - loss: 1.1893 - val_accuracy: 0.

In [12]:
# Evaluate the basic model
print("\n--- Evaluating Basic Model ---")
loss, accuracy = default_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy (Default Model): {accuracy:.4f}")
y_pred_default = np.argmax(default_model.predict(X_test), axis=1)
print("\nClassification Report (Default Model):")
print(classification_report(y_test, y_pred_default, target_names=le.classes_))


--- Evaluating Basic Model ---
Test Accuracy (Default Model): 0.8075
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Classification Report (Default Model):
              precision    recall  f1-score   support

           A       0.81      0.89      0.85       149
           B       0.71      0.82      0.76       153
           C       0.85      0.71      0.77       137
           D       0.82      0.80      0.81       156
           E       0.84      0.77      0.80       141
           F       0.77      0.84      0.81       140
           G       0.66      0.77      0.71       160
           H       0.79      0.56      0.66       144
           I       0.89      0.80      0.84       146
           J       0.79      0.86      0.82       149
           K       0.74      0.68      0.71       130
           L       0.82      0.83      0.83       155
           M       0.96      0.90      0.93       168
           N       0.90      0.85      0.87       151
           O       0.69      0.87     

In [17]:
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping

# Define the model for GridSearchCV
model_for_tuning = KerasClassifier(model=create_model, verbose=0)

# Define the grid of hyperparameters to search
param_grid = {
    'model__layers': [1, 2, 3],
    'model__neurons': [32, 64, 128],
    'model__activation': ['relu', 'tanh']
}

In [18]:
# Perform grid search with cross-validation
print("\n--- Hyperparameter Tuning with GridSearchCV ---")
grid_search = GridSearchCV(estimator=model_for_tuning, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)
grid_result = grid_search.fit(X_train, y_train)



--- Hyperparameter Tuning with GridSearchCV ---
Fitting 3 folds for each of 18 candidates, totalling 54 fits


C:\Users\vishu_pdk4f5i\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV] END model__activation=relu, model__layers=1, model__neurons=32; total time=   5.1s


C:\Users\vishu_pdk4f5i\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV] END model__activation=relu, model__layers=1, model__neurons=32; total time=   4.9s


...

C:\Users\vishu_pdk4f5i\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV] END model__activation=tanh, model__layers=3, model__neurons=128; total time=   7.3s


C:\Users\vishu_pdk4f5i\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV] END model__activation=tanh, model__layers=3, model__neurons=128; total time=   7.9s


C:\Users\vishu_pdk4f5i\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
# Summarize results
print("\nBest: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Best: 0.658562 using {'model__activation': 'tanh', 'model__layers': 3, 'model__neurons': 128}


In [21]:
# Train the best model found by grid search
print("\n--- Training Best Model ---")
best_model = grid_search.best_estimator_
history_best = best_model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
    verbose=1
)


--- Training Best Model ---
Epoch 1/100


C:\Users\vishu_pdk4f5i\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5483 - loss: 1.6470 - val_accuracy: 0.6944 - val_loss: 1.0983
Epoch 2/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7276 - loss: 0.9623 - val_accuracy: 0.7531 - val_loss: 0.8831
Epoch 3/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7757 - loss: 0.7670 - val_accuracy: 0.8031 - val_loss: 0.6939
Epoch 4/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8172 - loss: 0.6312 - val_accuracy: 0.8413 - val_loss: 0.5704
Epoch 5/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8443 - loss: 0.5297 - val_accuracy: 0.8400 - val_loss: 0.5380
Epoch 6/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8653 - loss: 0.4545 - val_accuracy: 0.8694 - val_loss: 0.4510
Epoch 7/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8806 - loss: 0.4071 - val_accuracy: 0.8913 - val_loss: 0.3993
Epoch 8/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8976 - loss: 0.3582 - val_accuracy: 0.8769

In [25]:
# Evaluate the tuned model
print("\n--- Evaluating Tuned Model ---")
# Get the model's accuracy using the score method
accuracy_tuned = best_model.score(X_test, y_test)
print(f"Test Accuracy (Tuned Model): {accuracy_tuned:.4f}")

# Get the predictions from the tuned model
y_pred_tuned = best_model.predict(X_test)

print("\nClassification Report (Tuned Model):")
print(classification_report(y_test, y_pred_tuned, target_names=le.classes_))


--- Evaluating Tuned Model ---
Test Accuracy (Tuned Model): 0.9635

Classification Report (Tuned Model):
              precision    recall  f1-score   support

           A       0.96      1.00      0.98       149
           B       0.94      0.92      0.93       153
           C       0.96      0.94      0.95       137
           D       0.95      0.97      0.96       156
           E       0.93      0.98      0.95       141
           F       0.97      0.94      0.96       140
           G       0.99      0.93      0.96       160
           H       0.92      0.88      0.90       144
           I       0.95      0.97      0.96       146
           J       0.98      0.97      0.98       149
           K       0.92      0.96      0.94       130
           L       0.99      0.97      0.98       155
           M       1.00      0.98      0.99       168
           N       0.97      0.97      0.97       151
           O       0.93      0.97      0.95       145
           P       0.96      

In [ ]:
'''
Evaluation and Discussion
The code evaluates both the default and the tuned models, printing a detailed classification report for each, which includes 

accuracy, precision, recall, and F1-score. The discussion below highlights the effects of hyperparameter tuning on model performance.
'''

In [ ]:
'''
Discussion of Results
The provided code first establishes a baseline with a simple ANN model using default hyperparameters. The classification report for this model
provides a starting point for performance measurement.
The hyperparameter tuning section  then employs 
GridSearchCV to explore a range of model configurations. The search identifies the optimal combination of hidden layers, neurons,
and activation functions that yields the highest cross-validation accuracy.
The final evaluation compares the performance of the best-tuned model against the default model. You will observe that the 
tuned model likely shows an improvement in key metrics such as accuracy, precision, recall, and F1-score. This demonstrates the importance 
of a systematic approach to hyperparameter tuning for enhancing the performance of neural networks. The early stopping callback in the final
training phase of the best model also helps prevent overfitting and ensures the model's robustness.
'''